## Assumption 1

    - Uniform distribution

    - Max 20

    - Frame 3, 5, 7


## Assumption 2

    - Uniform distribution

    - Max 80

    - Frame 3, 5, 7


## Assumption 3

    - Normal distribution

    - Max 40

    - Frame 3, 5, 7

In [5]:
import numpy as np
import matplotlib.pyplot as plt

FRAME_SIZE = 3
REF_LEN = 80
REF_MAX = 20

In [ ]:
class Page:
    def __init__(self):
        self.arrival_time = -1
        self.valid = True
        self.physical_address = -1

In [9]:
class RefString:
    def __init__(self, size, mx):
        self.size = size
        self.mx = mx

    def genUniform(self):
        return np.random.randint(1, self.mx, self.size)

    def genNormal(self, mean, std):
        return list(map(lambda x : int(x), np.random.normal(mean, std, self.size)))

[9, 3, 9, 2, 36, 14, 12, -7, 0, 15]


In [ ]:
class Fifo:
    def __init__(self, frame_size):
        self.frame_size = frame_size
        self.page = []
        self.page_fault = 0
        self.cur_time = 1
        for i in range(frame_size):
            self.page.append(Page())

    def find_space(self):
        for i in range(self.frame_size):
            if self.page[i].valid:
                return i
        return -1

    def find_page(self, addr):
        for i in range(self.frame_size):
            if self.page[i].physical_address == addr:
                return i
        return -1

    def replace(self):
        mn = self.cur_time
        space = 0
        for i in range(self.frame_size):
            if self.page[i].arrival_time < mn:
                mn = self.page[i].arrival_time
                space = i
        return space

    def run(self, ref_string):
        for string in ref_string:
            cur_index = self.find_page(string)
            if cur_index == -1:
                cur_index = self.find_space()
                if cur_index == -1:
                    cur_index = self.replace()
                    self.page_fault += 1
                self.page[cur_index].valid = False
                self.page[cur_index].physical_address = string
                self.page[cur_index].arrival_time = self.cur_time
                self.cur_time += 1

In [ ]:
class Lru:
    def __init__(self, frame_size):
        self.frame_size = frame_size
        self.page = []
        self.page_fault = 0
        self.cur_time = 1
        for i in range(frame_size):
            self.page.append(Page())

    def find_space(self):
        for i in range(self.frame_size):
            if self.page[i].valid:
                return i
        return -1

    def find_page(self, addr):
        for i in range(self.frame_size):
            if self.page[i].physical_address == addr:
                return i
        return -1

    def replace(self):
        mn = self.cur_time
        space = 0
        for i in range(self.frame_size):
            if self.page[i].arrival_time < mn:
                mn = self.page[i].arrival_time
                space = i
        return space

    def run(self, ref_string):
        for string in ref_string:
            cur_index = self.find_page(string)
            if cur_index == -1:
                cur_index = self.find_space()
                if cur_index == -1:
                    cur_index = self.replace()
                    self.page_fault += 1
                self.page[cur_index].valid = False
                self.page[cur_index].physical_address = string
            self.page[cur_index].arrival_time = self.cur_time
            self.cur_time += 1

In [ ]:
class Optimal:
    def __init__(self, frame_size):
        self.frame_size = frame_size
        self.page = []
        self.page_fault = 0
        self.cur_time = 1
        for i in range(frame_size):
            self.page.append(Page())

    def find_space(self):
        for i in range(self.frame_size):
            if self.page[i].valid:
                return i
        return -1

    def find_page(self, addr):
        for i in range(self.frame_size):
            if self.page[i].physical_address == addr:
                return i
        return -1

    def replace(self, ref_string, cur_index):
        mx = 0
        space = 0
        for i in range(self.frame_size):
            cur_mx = len(ref_string)
            for j in range(cur_index, len(ref_string)):
                if ref_string[j] == self.page[i].physical_address:
                    cur_mx = j
                    break
            if cur_mx > mx:
                mx = cur_mx
                space = i
        return space

    def run(self, ref_string):
        for (i, string) in enumerate(ref_string):
            cur_index = self.find_page(string)
            if cur_index == -1:
                cur_index = self.find_space()
                if cur_index == -1:
                    cur_index = self.replace(ref_string, i)
                    self.page_fault += 1
                self.page[cur_index].valid = False
                self.page[cur_index].physical_address = string
                self.page[cur_index].arrival_time = self.cur_time
                self.cur_time += 1

In [ ]:
ref = RefString(REF_LEN, REF_MAX)
string = ref.genUniform()
print(string)

fifo = Fifo(FRAME_SIZE)
fifo.run(string)
print(fifo.page_fault)

lru = Lru(FRAME_SIZE)
lru.run(string)
print(lru.page_fault)

opt = Optimal(FRAME_SIZE)
opt.run(string)
print(opt.page_fault)

In [ ]:
names = ['First Come First Serve', 'Shortest-Job-First', 'Round Robin']